In [1]:
import os
import pprint

# Define the target directory
target_directory = r"C:\Users\pablosal\Desktop\gbb-ai-hls-factory-prior-auth"  # change your directory here

# Check if the directory exists
if os.path.exists(target_directory):
    # Change the current working directory
    os.chdir(target_directory)
    print(f"Directory changed to {os.getcwd()}")
else:
    print(f"Directory {target_directory} does not exist.")

Directory changed to C:\Users\pablosal\Desktop\gbb-ai-hls-factory-prior-auth


### Understanding Semantic Kernel Components

Before diving into the implementation, let's understand the key components of Semantic Kernel.

#### The Kernel

The Kernel is the central orchestrator in Semantic Kernel. It manages:

- **Connecting to AI Models**: Interfaces with various AI models via connectors.
- **Registering and Invoking Plugins**: Manages the lifecycle and execution of plugins.
- **Managing Memory and Context**: Maintains state and context across interactions.
- **Interacting with the Planner**: Coordinates with the Planner to achieve specified goals.

#### Connectors

Connectors allow the Kernel to interface with various AI models and services. They define how the Kernel communicates with these models, whether they're:

- **OpenAI Models**: Such as GPT-3.5, GPT-4.
- **Azure OpenAI Services**: Leveraging Microsoft's cloud-based AI capabilities.
- **Local Models**: Using libraries like Hugging Face Transformers.

#### Plugins

Plugins are modular units that extend the Kernel's functionality. They consist of:

- **Prompt Functions**: Use natural language prompts to interact with AI models.
- **Native Functions**: Written in Python, performing deterministic tasks or interfacing with external services.

Plugins act as building blocks for complex workflows.

#### The Planner

The Planner uses AI to dynamically create a sequence of actions (a plan) to achieve a specified goal. It considers:

- **Available Plugins and Their Functions**: Understands the capabilities of each plugin.
- **Function Descriptions**: Uses metadata to understand what each function does.
- **Combining Functions**: Determines how functions can be combined to fulfill the goal.


### 🌟 Creating a Multi-Agent Architecture for Medical Research Documentation

In the realm of medical research, producing high-quality documentation is crucial. This guide aims to design a multi-agent system consisting of three specialized agents:

- **Medical Researcher**: Gathers and summarizes relevant medical information.
- **Clinical Evaluator**: Assesses the clinical relevance and accuracy of the information.
- **Medical Editor**: Refines the language and ensures adherence to medical writing standards.

Using Semantic Kernel's capabilities, we'll create plugins representing these agents and use a Planner to dynamically orchestrate their workflow based on a given goal.

By leveraging Semantic Kernel's capabilities, we can create a robust single-agent and sophisticated multi-agent system. Let's do it !

### Implementation

We will:

1. **Define Plugins**: Create plugins for the Medical Researcher, Clinical Evaluator, and Medical Editor.
2. **Configure the Kernel**: Set up the Kernel with the necessary connectors and plugins.
3. **Develop the Planner**: Implement the Planner to dynamically orchestrate the workflow.
4. **Execute the Plan**: Run the system to produce high-quality medical documentation.

In [2]:
#!pip install semantic-kernel --upgrade
import semantic_kernel as sk
print(f"We are using the following semantic_kernel library version: {sk.__version__}")

We are using the following semantic_kernel library version: 1.17.0


## 1. Define Plugins

We have defined the plugin for MedicalAgents located in the specified directory  `C:\Users\pablosal\Desktop\gbbai-agent-architecture-lab\src\plugins\plugins_store`

## 2. Configure The Kernel

In [3]:
import semantic_kernel as sk

# Initialize the kernel
kernel = sk.Kernel()

In [4]:
# add Azure OpenAI service connector to the kernel
import os
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

# Load environment variables from a .env file
from dotenv import load_dotenv
load_dotenv()

AZURE_OPENAI_KEY = os.getenv("AZURE_OPENAI_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_API_ENDPOINT")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_OPENAI_EMBEDDING_DEPLOYMENT = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT")
AZURE_OPENAI_CHAT_DEPLOYMENT_ID = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_ID")

service_id = "gpt-4o"
# Add Azure OpenAI chat completion
kernel.add_service(AzureChatCompletion(
    service_id=service_id,
    deployment_name=AZURE_OPENAI_CHAT_DEPLOYMENT_ID,
    api_key=AZURE_OPENAI_KEY,
    endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=AZURE_OPENAI_API_VERSION,
))


In [6]:
import os

# Define the parent directory and plugin name
parent_directory = os.path.abspath(os.path.join("src", "agenticai", "plugins", "plugins_store"))
plugin_name = "retrieval"

# Add the plugin to the kernel
plugin = kernel.add_plugin(parent_directory=parent_directory, plugin_name=plugin_name)

kernel.plugins

2024-12-18 19:40:26,113 - AIPolicyEvaluationPlugin - MainProcess - INFO     OpenAI client initialized successfully. (aievaluator.py:__init__:92)
[2024-12-18 19:40:26 - AIPolicyEvaluationPlugin:92 - INFO] OpenAI client initialized successfully.
2024-12-18 19:40:26,119 - AgenticRAG - Plugin AzureSearchPlugin - MainProcess - INFO     Initializing SearchClient with endpoint: https://search-ai-factory-centralus.search.windows.net, index_name: ai-policies-index (aisearch.py:__init__:49)
[2024-12-18 19:40:26 - AgenticRAG - Plugin AzureSearchPlugin:49 - INFO] Initializing SearchClient with endpoint: https://search-ai-factory-centralus.search.windows.net, index_name: ai-policies-index
2024-12-18 19:40:26,119 - AgenticRAG - Plugin AzureSearchPlugin - MainProcess - INFO     SearchClient initialized successfully. (aisearch.py:__init__:52)
[2024-12-18 19:40:26 - AgenticRAG - Plugin AzureSearchPlugin:52 - INFO] SearchClient initialized successfully.
2024-12-18 19:40:26,504 - AIQueryClassificationPlu

{'retrieval': KernelPlugin(name='retrieval', description=None, functions={'evaluate_search_results': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='evaluate_search_results', plugin_name='retrieval', description='Evaluates search results against a query to extract the most relevant policy paths.', parameters=[KernelParameterMetadata(name='query_text', description="The user's search query.", default_value=None, type_='str', is_required=True, type_object=<class 'str'>, schema_data={'type': 'string', 'description': "The user's search query."}, include_in_function_choices=True), KernelParameterMetadata(name='search_results', description='The list of search results to evaluate.', default_value=None, type_='list[dict[str, str]]', is_required=True, type_object=typing.List[typing.Dict[str, str]], schema_data={'type': 'array', 'items': {'type': 'object', 'additionalProperties': {'type': 'string'}}, 'description': 'The list of search results to evaluate.'}, include_in_function_cho

In [7]:
plugin.functions

{'evaluate_search_results': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='evaluate_search_results', plugin_name='retrieval', description='Evaluates search results against a query to extract the most relevant policy paths.', parameters=[KernelParameterMetadata(name='query_text', description="The user's search query.", default_value=None, type_='str', is_required=True, type_object=<class 'str'>, schema_data={'type': 'string', 'description': "The user's search query."}, include_in_function_choices=True), KernelParameterMetadata(name='search_results', description='The list of search results to evaluate.', default_value=None, type_='list[dict[str, str]]', is_required=True, type_object=typing.List[typing.Dict[str, str]], schema_data={'type': 'array', 'items': {'type': 'object', 'additionalProperties': {'type': 'string'}}, 'description': 'The list of search results to evaluate.'}, include_in_function_choices=True)], is_prompt=False, is_asynchronous=True, return_parameter=Kern

## 3. Develop the Planer

In [8]:
from semantic_kernel.connectors.ai.chat_completion_client_base import ChatCompletionClientBase
from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.azure_chat_prompt_execution_settings import (
    AzureChatPromptExecutionSettings,
)
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior

chat_completion : AzureChatCompletion = kernel.get_service(type=ChatCompletionClientBase)
execution_settings = AzureChatPromptExecutionSettings()
execution_settings.function_choice_behavior = FunctionChoiceBehavior.Auto() # the automatic planning loop https://learn.microsoft.com/en-us/semantic-kernel/concepts/planning?pivots=programming-language-python#the-automatic-planning-loop

## 4. Execute the Plan

In [27]:
from semantic_kernel.functions.kernel_arguments import KernelArguments
from semantic_kernel.contents.chat_history import ChatHistory
import logging
from semantic_kernel.utils.logging import setup_logging

# Create a history of the conversation
chat_history = ChatHistory()

# Add system message to the chat history to define the system's role
chat_history.add_system_message(
    "You are an AI-powered assistant specializing in producing high-quality medical research papers. "
    "Your goal is to assist users in creating comprehensive and accurate medical documents. "
    "You should generate relevant research, evaluate clinical data, and provide thorough evaluations to support the content."
)

# User message containing a detailed query
chat_history.add_user_message(
    "I would like to produce a high-quality medical document on the topic of 'The impact of AI on medical diagnostics.' "
    "Please provide detailed insights, key findings, and relevant research to support this topic. "
    "Additionally, include clinical evaluations and expert reviews to ensure the content is accurate and comprehensive."
)

# Set up logging
logging.basicConfig(
    format="[%(asctime)s - %(name)s:%(lineno)d - %(levelname)s] %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
setup_logging()
logging.getLogger("kernel").setLevel(logging.INFO)

# Get the response from the AI
result = (await chat_completion.get_chat_message_contents(
    chat_history=chat_history,
    settings=execution_settings,
    kernel=kernel,
    arguments=KernelArguments(input="The impact of AI on medical diagnostics"),
))[0]

# Print the result
print("AI Response:", result)

2024-12-18 18:22:23,012 - AIQueryClassificationPlugin - MainProcess - INFO     Classifying query: The impact of AI on medical diagnostics (aisearchclassification.py:classify_query:109)
[2024-12-18 18:22:23 - AIQueryClassificationPlugin:109 - INFO] Classifying query: The impact of AI on medical diagnostics
2024-12-18 18:22:23,014 - micro - MainProcess - INFO     Function generate_chat_response started at 2024-12-18 18:22:23 (aoai_helper.py:generate_chat_response:370)
[2024-12-18 18:22:23 - micro:370 - INFO] Function generate_chat_response started at 2024-12-18 18:22:23
2024-12-18 18:22:23,017 - micro - MainProcess - INFO     Sending request to Azure OpenAI at 2024-12-18 18:22:23 (aoai_helper.py:generate_chat_response:427)
[2024-12-18 18:22:23 - micro:427 - INFO] Sending request to Azure OpenAI at 2024-12-18 18:22:23
2024-12-18 18:22:23,358 - micro - MainProcess - INFO     Function generate_chat_response finished at 2024-12-18 18:22:23 (Duration: 0.34 seconds) (aoai_helper.py:generate_ch

AI Response: ### The Impact of AI on Medical Diagnostics

Artificial Intelligence (AI) is revolutionizing various industries, with healthcare being a prominent field witnessing significant advancements. The application of AI in medical diagnostics holds transformative potential, enhancing the precision, efficiency, and accessibility of diagnostic services. This document explores the detailed insights, key findings, and relevant research supporting the impact of AI on medical diagnostics, along with clinical evaluations and expert reviews to ensure comprehensive and accurate content.

#### Introduction to AI in Medical Diagnostics
AI leverages machine learning algorithms, natural language processing, and advanced data analytics to assist healthcare professionals in diagnosing diseases. These technologies can analyze complex medical data, recognize patterns, and provide accurate predictions, thereby augmenting the diagnostic process.

#### Key Findings and Insights

1. **Enhanced Diagnos